# Möbius-Digit‑Thread **Grid Explorer (v13)** — real‑safe damping

This notebook rebuilds the full pipeline from scratch so that fractional
powers never generate complex numbers and all rules we agreed on are explicit
and tweak‑ready.

**Rule baseline**

| knob | value |
|------|-------|
| bright tail | 400 ones |
| gap construction | prime gaps, `gap_scale = 20` |
| spacing | every 3rd dark pulse |
| φ-micro grid | `0.003944 ± 1×10⁻⁷` (9 pts) |
| tiny exponent k | `{3.8, 4.0, 4.2, 4.3, 4.4, 4.5}` |
| damped breath λ | `{None(=1), 0.2, 0.3, 0.4, 0.5}` |
| bits | 64 / 128 / 256 |

Diagnostics reproduced:

* digit‑thread finder (π, e, φ, γ, √2) ≥4 consecutive
* power‑of‑3 proximity (≤ 0.5 %)
* Fibonacci closeness check (Δx < 10⁻⁵ to 64‑ and 128‑bit seeds)

The final grid search prints the best 20 rows sorted by `rel_err`.
Auto‑prints any hit that breaks **1 %** and shouts if it beats **0.1 %**.

In [1]:

import mpmath as mp, pandas as pd, itertools, math
mp.mp.dps = 260   # high precision

ALPHA_INV = mp.mpf('137.035999084')
ALPHA      = 1/ALPHA_INV
PHI        = (1+mp.sqrt(5))/2


In [2]:

def digits_of(num, n=700):
    return str(num).replace('.','')[:n]
CONST = { 'pi':digits_of(mp.pi),
          'e' :digits_of(mp.e),
          'phi':digits_of(PHI),
          'gamma':digits_of(mp.euler),
          'sqrt2':digits_of(mp.sqrt(2)) }


In [3]:

def primes_upto(n=30000):
    sieve=[True]*(n+1); sieve[0:2]=[False,False]
    for i in range(2,int(mp.sqrt(n))+1):
        if sieve[i]:
            sieve[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [i for i,v in enumerate(sieve) if v]

PRIMES=primes_upto()
def prime_gaps(cnt): return [PRIMES[i+1]-PRIMES[i] for i in range(cnt)]


In [4]:

def build_tail(bright_len=400, spacing=3, gap_scale=20):
    gaps=[round(gap_scale/4) for g in prime_gaps(bright_len//spacing)]
    tail=[1]
    gi=iter(gaps)
    for i in range(bright_len):
        tail.append(1)
        if (i+1)%spacing==0:
            tail.append(next(gi,1))
    return tail

def cf_to_real(coeffs):
    v=mp.mpf(0)
    for a in reversed(coeffs): v=1/(a+v)
    return v


In [5]:

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_fraction(bits):
    s=mp.mpf(0)
    for i,b in enumerate(bits):
        if b=='1':
            s += mp.power(2,-(i+1))
    return s

F64  = fib_fraction(fib_word_bits(64))
F128 = fib_fraction(fib_word_bits(128))


In [6]:

def breath_once(x, tiny, phi):
    x = phi*x
    x -= tiny
    x = 1/x/9
    x -= tiny
    x = x/9
    return x

def damped_breath_real(x, tiny, phi, lam=None):
    x = breath_once(x,tiny,phi)
    if lam is None: return x
    sign = mp.sign(x)
    x = sign * mp.power(abs(x), lam)
    return x


In [7]:

phi_centre = mp.mpf('0.003944')
phi_vals = [phi_centre + i*1e-7 for i in range(-4,5)]
k_vals   = [3.8,4.0,4.2,4.3,4.4,4.5]
lam_vals = [None,0.2,0.3,0.4,0.5]

rows=[]
tail = build_tail(spacing=3, gap_scale=20)
seed0 = cf_to_real(tail)

for bits in (64,128,256):
    for k in k_vals:
        tiny = ALPHA_INV / mp.power(10,k)
        for phi in phi_vals:
            for lam in lam_vals:
                x = damped_breath_real(seed0+tiny, tiny, phi, lam)
                rel = float(abs((x-ALPHA)/ALPHA))
                rows.append(dict(bits=bits,k=k,phi=float(phi),lam=lam,rel_err=rel,
                                 fib64_near=abs(x-F64)<=1e-5,
                                 fib128_near=abs(x-F128)<=1e-5))
                if rel < 0.01 and lam is not None:
                    print(f"Hit <1% | bits {bits} k {k} phi {phi} λ {lam} rel {rel:.6g}")
                if rel < 0.001:
                    print("""*** Hit <0.1% ***""")
df = pd.DataFrame(rows).sort_values('rel_err').head(40)
print("Top 40 by rel_err:")
display(df)


Top 40 by rel_err:


,bits,k,phi,lam,rel_err,fib64_near,fib128_near
0,64,3.8,0.003944,NaN,89.254495,False,False
270,128,3.8,0.003944,NaN,89.254495,False,False
540,256,3.8,0.003944,NaN,89.254495,False,False
275,128,3.8,0.003944,NaN,89.254782,False,False
545,256,3.8,0.003944,NaN,89.254782,False,False
5,64,3.8,0.003944,NaN,89.254782,False,False
550,256,3.8,0.003944,NaN,89.255069,False,False
10,64,3.8,0.003944,NaN,89.255069,False,False
280,128,3.8,0.003944,NaN,89.255069,False,False
15,64,3.8,0.003944,NaN,89.255356,False,False
